In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
random.seed(0)
torch.manual_seed(0)

device = torch.device("mps")

learning_rate = .001
training_epochs = 15
batch_size = 100

In [16]:
mnist_train = dsets.FashionMNIST(root='dset/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.FashionMNIST(root='dset/', train=False, transform=transforms.ToTensor(), download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

100%|██████████| 26.4M/26.4M [00:06<00:00, 4.36MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 84.4kB/s]
100%|██████████| 4.42M/4.42M [00:07<00:00, 592kB/s] 
100%|██████████| 5.15k/5.15k [00:00<00:00, 7.12MB/s]


In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # L1 Image size = (?, 1, 28, 28)
        # L1 Conv       = (?, 32, 28, 28)
        # L1 Pool       = (?, 32, 14, 14)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # L2 Conv       = (?, 64, 14, 14)
        # L2 Pool       = (?, 64, 7, 7)
        # L2 Output     = (?, 64, 7, 7)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=1)
        )
        # L3 Conv       = (?, 64, 7, 7)
        # L3 Pool       = (?, 64, 5, 5)
        # L3 Output     = (?, 64, 5, 5)
        self.fc = torch.nn.Linear(3 * 3 * 64, 10, bias=True)
        # FC            = (?, 10)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [18]:
model = CNN().to(device)

In [19]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.540456474
[Epoch:    2] cost = 0.338510036
[Epoch:    3] cost = 0.293289214
[Epoch:    4] cost = 0.266239434
[Epoch:    5] cost = 0.247671545
[Epoch:    6] cost = 0.227959499
[Epoch:    7] cost = 0.213977009
[Epoch:    8] cost = 0.201440439
[Epoch:    9] cost = 0.190190554
[Epoch:   10] cost = 0.175933644
[Epoch:   11] cost = 0.167948306
[Epoch:   12] cost = 0.159854844
[Epoch:   13] cost = 0.149623096
[Epoch:   14] cost = 0.138035133
[Epoch:   15] cost = 0.132255346
Learning Finished!


In [21]:
accuracy = 0
with torch.no_grad():
    for X_test, Y_test in data_loader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy += correct_prediction.float().sum()
    accuracy = accuracy.item() / len(mnist_train)
print('Accuracy: ', accuracy)

Accuracy:  0.9606666666666667
